In [ ]:

# Phase Unwrapping Goldstein (Orientation Unwrapping)
import queue
import imageio


#testimage\n",
# img = imageio.imread('D:/Levchenko_Project_Readings/Pictures_of_fingerprints/Original_fingerprint.jpg')\n",
#convert to grayscale\n"
import cv2
img = cv2.imread('D:/Levchenko_Project_Readings/Pictures_of_fingerprints/Original_fingerprint.jpg', cv2.IMREAD_GRAYSCALE)
    
# img = imageio.imread('D:/Levchenko_Project_Readings/Pictures_of_fingerprints/Original_fingerprint_gray.jpg')
# print(\"gray image is read\")
  
    

# Calculates residue value from current, right, down, left
def check_residue(pixel,phase_image):

    if (pixel[0] >= len(phase_image)-1 or pixel[1] >= len(phase_image[0])-1):
        return None

    def sign(x):
        return int(1-(x<=0))
    
    #   check for residue
    residue = int(0)

#         print(pixel)

    if( abs(-phase_image[pixel[0]][pixel[1]] + phase_image[pixel[0]][pixel[1]+1])> 0.5 ):
        residue += sign(-phase_image[pixel[0]][pixel[1]] + phase_image[pixel[0]][pixel[1]+1])

    if(abs(phase_image[pixel[0]+1][pixel[1]+1] - phase_image[pixel[0]][pixel[1]+1])>0.5):
        residue += sign(phase_image[pixel[0]+1][pixel[1]+1] - phase_image[pixel[0]][pixel[1]+1])

    if(abs(phase_image[pixel[0]+1][pixel[1]] -phase_image[pixel[0]+1][pixel[1]+1])>0.5):
        residue += sign(phase_image[pixel[0]+1][pixel[1]] -phase_image[pixel[0]+1][pixel[1]+1])

    if (abs(phase_image[pixel[0]][pixel[1]]  - phase_image[pixel[0]+1][pixel[1]])>0.5):
        residue += sign(phase_image[pixel[0]][pixel[1]]  - phase_image[pixel[0]+1][pixel[1]])

    print("residue is ")
    print(residue%1)
    return residue%1    
    
    
    
    
def Goldstein_unwrap(image ,ridge_freq):

   # Filtering with triangulation and ridge frequency (implement later)



    new_image = image


    # Here marked for branch element is [visited,charge]
    marked_for_branch = []
    zero_vec = []

    for j in range(0,len(new_image[0])):
        zero_vec.append([0,0])
    for i in range(0,len(new_image)):
        marked_for_branch.append(zero_vec)


    # Branch Cut Portion Using Priority Queue
    cuts_array = []
    initial_box_size = 3
    added_box_size = 0
    queue1 = queue.Queue(maxsize=0)
    queue1.put([0,0])
    residue = None

    print("gets to this checkpoint")

#interferogram scanned until residue found

#     residue = [0,0]



    #Branch cut algorithm
    def cutting(residue, queue1,added_box_size):        
        print("calls function")
        residuevalue = 0
        residuefound = False

        # Scans until residue is found by adding elements to a queue until residue is found
        while(residuefound == False):
            print("in loop")

            # check if there is an input residue if not start at the element popped in the list
            if (residue == None):
                #Pop the current element in the queue
                current = queue1.get()
                # change the residue to be the popped element for when exiting the loop
                residue = current
            #if not then empty the list and run the algorithm with current as assigned
            else:
#                 queue1 = queue.Queue(maxsize=0)
                current = residue

#             print(check_residue(previous,new_image))

            #Update the residue value and find out if it is found
            print(residue)    
            residuevalue = check_residue(current,new_image)
            residuefound = bool( residuevalue != 0 )

            # place the other 3 in the queue\n",
            if(current[0] <= len(new_image)-2 and current[0] <= len(new_image[0])-2):
                print("gets qued")
                print([current[0]+1,current[1]])
                queue1.put([current[0]+1,current[1]])
                queue1.put([current[0]+1,current[1]+1])
                queue1.put([current[0],current[1]+1])

            #empty queue once found
            queue1 = queue.Queue(maxsize=0)

            # Jumps out of loop once the residue is found
            print("found")
            print(residuevalue)

            # Place a box of size 3 + added length around the residue and search for another residue
            for boxlen in range(1+int(added_box_size/2), initial_box_size+added_box_size):
                for boxwidth in range(1+int(added_box_size/2), initial_box_size+added_box_size):

                    #checks if the indexes are valid this is the problem?
                    try:
                        getresidue = check_residue([residue[0]-1-int(added_box_size/2)+boxlen,residue[1]-1-int(added_box_size/2)+boxwidth],new_image)
                    except:
                        continue
                    
                    print("this is the cuts_array")
                    print(cuts_array)

                    if ([residue,getresidue,0] in cuts_array or [residue,getresidue,1] in cuts_array):
                        continue
                    # if residue is found 
                    print("find other residue")
                    print(getresidue)
#                     print(boxlen != 1+int(added_box_size/2))
#                     print(boxwidth !=1+int(added_box_size/2))



#                     intermed = !(boxlen ==1+int(int(added_box_size)/2) and boxwidth ==1+int(int(added_box_size)/2)   )
                    if getresidue != 0:
#     and intermed :
                        # current problem, never jumps into this statement





                        print("passes criterion")

                        # If residue has opposite sign
                        if(residuevalue*getresidue< 0):
                            print("residue not same sign")
                            # locates the cut and designates as uncharged(0)
                            cuts_array.append([residue,getresidue, 0])
                            cutting([residue[0]-1-int(added_box_size/2)+boxlen,residue[1]-1-int(added_box_size/2)+boxwidth],queue1,added_box_size)


                        # If residue has same sign
                        if(residuevalue*getresidue > 0):
                            print("residue same sign")
                            # make the cut indicating it is charged
                            cuts_array.append([residue,getresidue, 1])

                            #Essentially a repeat of what happens above but there is a difference in calling and only cares about the negative sign
                            # box is moved to new residue and search continues until opposite residue is located 
                            for boxlen in range(1+int(added_box_size/2), initial_box_size+added_box_size):
                                for boxwidth in range(1+int(added_box_size/2), initial_box_size+added_box_size):
                                    getresidue = check_residue([residue[0]-1-int(added_box_size/2)+boxlen,residue[1]-1-int(added_box_size/2)+boxwidth],new_image)

                                    if ([residue,getresidue,0] in cuts_array or [residue,getresidue,1] in cuts_array):
                                        continue
                                    # if residue is found 
#                                     intermed2 = !(boxlen ==1+int(int(added_box_size)/2) and boxwidth ==1+int(int(added_box_size)/2))
                                    if getresidue != 0:
#         and intermed2:\n",
                                        # If residue has opposite sign
                                        if(residuevalue*getresidue< 0):
                                            print("residue not same sign")
                                            # locates the cut and designates as uncharged(0)
                                            cuts_array.append([residue,getresidue, 0])
                                            cutting(None,queue1,added_box_size)


                            # If no more residues are left     
                            #postincrement and run on the current pixel
                            added_box_size+=2
                            if ( added_box_size+3 == min(len(image),len(image[0])) ):
                                                  return
                            cutting([residue[0]-1-int((added_box_size-2)/2)+boxlen,residue[1]-1-int((added_box_size-2)/2)+boxwidth],queue1,added_box_size)

    cutting(residue,queue1,added_box_size) 
    print(cuts_array)
    return

#     print(\"function call is made\")            

    # Integration of phase unwrapping avoiding the branch cuts
#     unwrap_queue = Queue()

#     unwrap_queue.put([0,0])



Goldstein_unwrap(img ,4)
  


  
# # 8-bit Grayscale convert
# # function phi is the composite phase
    
# def grayscale_8(x,y):
#     return 255*(1+cos(phi(x,y)))/2

#     # Reconstruct the Orientation field
    
# # block_dim = 8
    
# #     def S
    
#     # m,n are coordinates for block, weighting function is applied to calculate the directions
# def orient(m,n,weighting_function):

#     #if there are singular minutia, use this as part of the function
#     #Singular point influence
#     # core 1, delta -1 index
#     D_s = 0
#     if (minutia_array["contains singular point index"] == True):
#         for z in (minutia_array):
#             if (z["singular point index"] == True):
#                 D_s += (1/2)*z["core delta index"]*arctan((n-minutia_array["n_si range of y in 8 by 8 block"])/(m-minutia_array["m_si range of x in 8 by 8 block"]))



#     # This is for the non-singularity minutia
#     u = 0
#     for i in range(0, len(minutia_array)):
#         u += cos(2*minutia_array["index for direction"]) * weighting_function(minutia_array["relevant for weighting parts"])

#     v = 0
#     for j in range(0, len(minutia_array)):
#         v += cos(2*minutia_array["index for direction"]) * weighting_function(minutia_array["relevant for weighting parts"])

#     return D_s + (1/2)*arctan(v/u)

#     # Estimate Gradient of Continuous Phase
  
       
# # Phase estimation
# Phase = dict()


# # Define Gradient phase at block (m,n)
# def grad(m,n, ridge_freq):
#     return 2*pi*ridge_freq*exp(1j*(orient(m,n,weighting_function)+pi/2))

# def Spiral_Grad(m,n,ridge_freq):\n",
#     #if there are singular minutia, use this as part of the function
#     #Singular point influence
#     Spiral = 0
#     if (minutia_array["contains singular point index"] == True):
#         for z in (minutia_array):
#             if (z["singular point index"] == True):
#                 Spiral += z["residue_sign"]*arctan((n-minutia_array["n_si range of y in 8 by 8 block"])/(m-minutia_array["m_si range of x in 8 by 8 block"]))


# def Cont_Grad(m,n,ridge_freq):
#     grad(m,n,ridge_freq) - Spiral_Grad(m,n,ridge_freq)


# def Cont_Phase(x,y):
#     return Cont_Grad(m,n,ridge_freq).real*x + Cont_Grad(m,n,ridge_freq).imag*y + Phase[(m,n)]


# def phase_estimate_border(image,x,y,m_prev,n_prev,m,n):
#     return Cont_Grad(m_prev,n_prev,ridge_freq).real*x + Cont_Grad(m_prev,n_prev,ridge_freq).imag*y -(Cont_Grad(m,n,ridge_freq).real*x + Cont_Grad(m,n,ridge_freq).imag*y ) + Phase[(m_prev,n_prev)]

# def phase_estimate(m,n):
#     estimate = 0
#     for i in range(0,8):
#         estimate += phase_estimate_border(image,m(topleft_x)+i,n(topleft_y),m,n-1,m,n)
#     for j in range(0,8):
#         estimate += phase_estimate_border(image,m(topleft_x)+7,n(topleft_y)+j,m-1,n,m,n)
#         estimate += phase_estimate_border(image,m(topleft_x)+7,n(topleft_y)+j,m+1,n,m,n)
#     for k in range(0,8):
#         estimate += phase_estimate_border(image,m(topleft_x)+k,n(topleft_y)+7,m,n+1,m,n)


#     return estimate


